In [ ]:
import random
import compyute as cp

In [ ]:
device = "cuda" if cp.engine.gpu_available() else "cpu"
device

# Example 4

### Convolutional Neural Network: LeNet & MNIST

The goal of this model is to classify images of hand-written digits.

### Step 1: Prepare data
You will need to download the dataset from https://www.kaggle.com/competitions/digit-recognizer/data and place it into the *data* directory. Only using the official training data for training, validation and testing, since it is just to showcase the framework.

In [ ]:
# ! pip install pandas

In [ ]:
import pandas as pd

data = pd.read_csv('../data/mnist/train.csv')
data.head()

In [ ]:
tensor = cp.tensor(data.to_numpy())
train, val, test = cp.preprocessing.split_train_val_test(tensor, ratio_val=0.1, ratio_test=0.1)

In [ ]:
X_train, y_train = train[:, 1:], train[:, 0].to_int()
X_val, y_val = val[:, 1:], val[:, 0].to_int()
X_test, y_test = test[:, 1:], test[:, 0].to_int()

In [ ]:
X_train = cp.reshape(X_train, shape=(X_train.shape[0], 1 , 28, -1)).to_float()
X_val = cp.reshape(X_val, shape=(X_val.shape[0], 1, 28, -1)).to_float()
X_test = cp.reshape(X_test, shape=(X_test.shape[0], 1, 28, -1)).to_float()

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_val.shape=}')
print(f'{y_val.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

Scaling

In [ ]:
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

### Step 2: Build the neural network structure

In [ ]:
import compyute.nn as nn

# LeNet
model = nn.Sequential(
    nn.Convolution2DBlock(1, 6, kernel_size=5, activation="sigmoid", padding="same"),
    nn.AvgPooling2D(kernel_size=2),

    nn.Convolution2DBlock(6, 16, kernel_size=5, activation="sigmoid", padding="valid"),
    nn.AvgPooling2D(kernel_size=2),
    
    nn.Flatten(),
    nn.DenseBlock(16*5*5, 120, activation="sigmoid"),
    nn.DenseBlock(120, 84, activation="sigmoid"),
    nn.Linear(84, 10)
)

model.to_device(device)

In [ ]:
summary = model.get_summary(input_shape=(1, 28, 28))
print(summary)

### Step 3: Train the model

In [ ]:
from compyute.nn.trainer import Trainer
from compyute.nn.trainer.callbacks import History, ProgressBar

history = History()

trainer = Trainer(
    model=model,
    optimizer=nn.optimizers.Adam(),
    loss=nn.CrossEntropy(),
    metric=nn.Accuracy(),
    callbacks=[
        history,
        ProgressBar(mode="step"),
    ]
)

In [ ]:
epochs = 10
batch_size = 128

trainer.train(X_train, y_train, epochs=epochs, batch_size=batch_size, val_data=(X_val, y_val))

In [ ]:
# ! pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt

def plot_history(t1, t2):
    trace1 = history[t1]
    trace2 = history[t2]
    plt.figure(figsize=(10, 3))
    plt.plot(cp.arange(start=1, stop=len(trace1) + 1), trace1, linewidth=1)
    plt.plot(cp.arange(start=1, stop=len(trace2) + 1), trace2, linewidth=1)

plot_history("loss", "accuracy_score")

### Step 4: Evaluate the model

In [ ]:
loss, accuracy = trainer.evaluate_model(X_test, y_test, batch_size)
print(f'loss {loss:.4f}')
print(f'accuracy {accuracy*100:.2f}')

In [ ]:
# ! pip install scikit-learn

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy
from compyute.nn.utils import batched

y_pred = batched(model, batch_size=batch_size, device=model.device, shuffle_data=False)(X_test)
probs, _ = nn.functional.softmax(y_pred)

cm = confusion_matrix(
    y_true=y_test.to_numpy(),
    y_pred=cp.argmax(probs, axis=-1).to_numpy(),
    labels=cp.unique(y_test).to_numpy()
)

r = cp.arange(10).to_numpy()
plt.imshow(cm, cmap="Blues")
plt.xlabel("prediction")
plt.ylabel("target")
plt.xticks(ticks=r, labels=r)
plt.yticks(ticks=r, labels=r)
for (j, i), label in numpy.ndenumerate(cm):
    plt.text(i, j, str(int(label)), ha="center", va="center")

### Step 5: Explore the inner workings
Pick a random image from the testing dataset.

In [ ]:
i = random.randint(0, len(X_test) - 1)
image = cp.moveaxis(X_test[i], from_axis=0, to_axis=-1)  # matplotlib needs the color channel to be the last dim

plt.figure(figsize=(3, 3))
plt.imshow(image.data, cmap='gray')
plt.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

Use it to predict a number and show the probability distribution of the outcome.

In [ ]:
print(f"correct label: {y_test[i].item()}")

image_tensor = X_test[None, i].to_device(device)

# retain values so we can look at intermediates
with model.do_retain_values():
    logits = model(image_tensor)

probs = cp.squeeze(cp.nn.functional.softmax(logits)[0])
pred = cp.squeeze(cp.argmax(probs, axis=-1)).item()

print(f"predicted label: {pred}")

plt.figure(figsize=(5, 3))
plt.xticks(ticks=r)
plt.bar(r, probs.to_numpy())
plt.xlabel("class")
plt.ylabel("probability");

Every layer of the model can be accessed to explore their output. Here we iterate over all the kernels of the convolutional layer to explore what they learned to focus on in images.

In [ ]:
def plot_channels(array, label):
    plt.figure(figsize=(20, 20))
    for i in range(array.shape[0]):
        plt.subplot(10, 8, i + 1)
        image = array[i, :, :]
        plt.imshow(image, vmin=cp.min(image).item(), vmax=cp.max(image).item(), cmap="gray")
        plt.xlabel(f"channel {str(i + 1)}")
        plt.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
    plt.show()

In [ ]:
conv1 = model.modules[0].modules[0]

out = conv1.y[0].to_cpu()
out_min = cp.min(out, axis=0)
out_max = cp.max(out, axis=0)
out = (out - out_min) / (out_max - out_min)
plot_channels(out, "channel")

In [ ]:
conv2 = model.modules[2].modules[0]

out2 = conv2.y[0].to_cpu()
out_min2 = cp.min(out2, axis=0)
out_max2 = cp.max(out2, axis=0)
out2 = (out2 - out_min2) / (out_max2 - out_min2)
plot_channels(out2, "channel")

Learned filters

In [ ]:
weights1 = cp.sum(conv1.w, axis=1).to_cpu()
weights_min1 = cp.min(weights1, axis=0)
weights_max1 = cp.max(weights1, axis=0)
weights1 = (weights1 - weights_min1) / (weights_max1 - weights_min1)
plot_channels(weights1, "filter")

In [ ]:
weights2 = cp.sum(conv2.w, axis=1).to_cpu()
weights_min2 = cp.min(weights2, axis=0)
weights_max2 = cp.max(weights2, axis=0)
weights2 = (weights2 - weights_min2) / (weights_max2 - weights_min2)
plot_channels(weights2, "filter")